# Tratando valores nulos

## Valores anteriores
Rescatando los valores trabajados en la unidad anterior...

In [39]:
import pandas as pd

url = "https://gist.githubusercontent.com/ahcamachod/a572cfcc2527046db93101f88011b26e/raw/ffb13f45a79d31223e645611a119397dd127ee3c/alquiler.csv"

pd.read_csv(url)

datos = pd.read_csv(url, delimiter=";")
df_tipo_precio = datos.groupby("Tipo")[["Valor"]].mean().sort_values("Valor")
inmuebles_comerciales = [
    "Conjunto Comercial/Sala",
    "Edificio Completo",
    "Tienda/Salón",
    "Casa Comercial",
    "Terreno Estándar",
    "Cochera/Estacionamiento",
    "Galpón/Depósito/Almacén",
    "Tienda en Centro Comercial",
    "Hotel",
    "Loteo/Condominio",
    "Industria",
]
df = datos.query("@inmuebles_comerciales not in Tipo")
df_tipo_precio = df.groupby("Tipo")[["Valor"]].mean().sort_values("Valor")
porcentaje_tipo = df.Tipo.value_counts(normalize=True).to_frame()
df_tipos_departamentos = df.query('Tipo == "Departamento"')


In [40]:
# Al devolver isnull un booleano (equivalente a 0 o 1) se permitirá generar una suma, que mostrará si existen valores nulos o no
df_tipos_departamentos.isnull().sum()

Tipo               0
Colonia            0
Habitaciones       0
Garages            0
Suites             0
Area               0
Valor              7
Condominio       493
Impuesto        3797
dtype: int64

## Tratamiento de datos nulos
Equipo de machine learning indicó que a los datos que contengan datos nulos, se les deberá asignar valor 0

In [41]:
# Método para rellenar NA/NaN con el valor indicado como argumento
df_tipos_departamentos.fillna(0)

,Tipo,Colonia,Habitaciones,Garages,Suites,Area,Valor,Condominio,Impuesto
3,Departamento,Centro Histórico,1,0,0,15,2800.0,1365.0,70.0
4,Departamento,Del Valle,1,0,0,48,2800.0,805.0,0.0
7,Departamento,Centro Histórico,1,0,0,36,4200.0,0.0,0.0
8,Departamento,Condesa,1,0,1,40,7000.0,1963.5,175.0
10,Departamento,Condesa,4,3,1,243,45500.0,7000.0,2810.5
...,...,...,...,...,...,...,...,...,...
25111,Departamento,Lomas de Chapultepec,3,0,1,80,10500.0,3535.0,871.5
25113,Departamento,Coyoacán,3,1,2,150,52500.0,4900.0,2100.0
25114,Departamento,Narvarte,1,0,0,28,3500.0,1260.0,87.5
25118,Departamento,Mixcoac,2,1,0,48,4900.0,1781.5,129.5


In [42]:
# Habrá que almacenar el cambio en el dataframe
df_tipos_departamentos = df_tipos_departamentos.fillna(0)

# Comprobando existencia de nulos
df_tipos_departamentos.isnull().sum()

Tipo            0
Colonia         0
Habitaciones    0
Garages         0
Suites          0
Area            0
Valor           0
Condominio      0
Impuesto        0
dtype: int64

# Remover registros inconsistentes
Algunos registros de la base de datos no tienen mucho sentido, por ejemplo:
- Deptos con valor de alquiler igual a 0
- Deptos con valor de condominio igual a 0

Al ser inconsistentes será conveniente eliminarlos de la base de datos

In [43]:
# Seleccionando a través de un query lo pedido
df_tipos_departamentos.query("Valor == 0 | Condominio == 0")

,Tipo,Colonia,Habitaciones,Garages,Suites,Area,Valor,Condominio,Impuesto
7,Departamento,Centro Histórico,1,0,0,36,4200.0,0.0,0.0
49,Departamento,Santa Fe,2,1,1,70,0.0,3395.0,238.0
74,Departamento,Ciudad de los Deportes,2,0,0,80,3500.0,0.0,0.0
116,Departamento,El Rosedal,2,1,0,50,3850.0,0.0,0.0
136,Departamento,Santa Fe,1,1,1,65,8225.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
24721,Departamento,Doctores,2,0,0,43,3150.0,0.0,56.0
24736,Departamento,Condesa,1,0,0,30,9100.0,0.0,0.0
24866,Departamento,Condesa,3,0,0,130,16100.0,0.0,0.0
24892,Departamento,Loma Bonita,1,0,0,40,1925.0,0.0,0.0


In [44]:
df_tipos_departamentos.query("Valor == 0 | Condominio == 0").index

Index([    7,    49,    74,   116,   136,   185,   199,   432,   504,   527,
       ...
       24426, 24555, 24563, 24597, 24671, 24721, 24736, 24866, 24892, 25014],
      dtype='int64', length=500)

Se utilizarán los índices para poder eliminar el registro completo, fila por fila.

In [45]:
# Se utilizará como filtro
df_remover = df_tipos_departamentos.query("Valor == 0 | Condominio == 0").index

In [46]:
# Axis = 0 se refiere a las filas, dónde estará cada uno de los índices buscados
# Inplace es para no tener que reasignar la variable al valor filtrado
df_tipos_departamentos.drop(df_remover, axis=0, inplace=True)

In [47]:
# Comprobando eexistencia de valores o condominios = 0
df_tipos_departamentos.query("Valor == 0 | Condominio == 0").index

Index([], dtype='int64')

In [48]:
df_tipos_departamentos.Tipo.unique()

array(['Departamento'], dtype=object)

Como solo estamos trabajando sobre los departamentos, se optará por eliminar la columna 'Tipo' porque en este caso no tiene sentido mantenerla.

In [49]:
# Axis ahora es 1 porque estará apuntando a una columna y no a filas
df_tipos_departamentos.drop('Tipo', axis=1, inplace=True)

In [50]:
df_tipos_departamentos.sample(10)

,Colonia,Habitaciones,Garages,Suites,Area,Valor,Condominio,Impuesto
18408,Narvarte,3,2,1,110,11550.0,3850.0,3.5
4191,Santa Fe,2,1,1,78,3150.0,4200.0,546.0
3282,Narvarte,3,0,0,80,8750.0,1575.0,227.5
357,Coyoacán,3,1,1,145,25200.0,10150.0,1750.0
9046,Coyoacán,3,2,0,150,23100.0,7700.0,1354.5
8211,Bosques de las Lomas,2,1,1,70,9100.0,5439.0,815.5
7652,Coyoacán,3,1,1,280,43750.0,7798.0,2135.0
12025,Condesa,4,0,1,200,15750.0,6786.5,11550.0
5604,Bosques de las Lomas,2,0,0,55,7700.0,3080.0,437.5
1194,Del Valle Centro,1,1,0,44,1750.0,1211.0,0.0


# Aplicar filtros
En este momento, el equipo de ML desea evaluar solo propiedades en escenarios específicos, como:

- Apartamentos que tienen 1 dormitorio y un alquiler menor a MXN 4200.
- Apartamentos que tienen al menos 2 dormitorios, un alquiler menor a MXN 10500 y una superficie mayor a 70 m².

Con base en esto, necesitamos aplicar los filtros necesarios para obtener esta información específica.

In [63]:
df = df_tipos_departamentos
# 1. apts con 1 dormitorio y alquiler menor a 4200

# crea serie pandas con booleanos
seleccion1 = df['Habitaciones'] == 1
seleccion1

3         True
4         True
8         True
10       False
13       False
         ...  
25111    False
25113    False
25114     True
25118    False
25119    False
Name: Habitaciones, Length: 14264, dtype: bool

In [64]:
# El aplicar este filtro se llama máscara
df[seleccion1]

,Colonia,Habitaciones,Garages,Suites,Area,Valor,Condominio,Impuesto
3,Centro Histórico,1,0,0,15,2800.0,1365.0,70.0
4,Del Valle,1,0,0,48,2800.0,805.0,0.0
8,Condesa,1,0,1,40,7000.0,1963.5,175.0
19,Narvarte,1,0,0,48,2625.0,2702.0,87.5
37,Centro Histórico,1,0,0,27,2625.0,1298.5,66.5
...,...,...,...,...,...,...,...,...
25090,Coyoacán,1,0,0,55,7000.0,2205.0,66.5
25092,Santa Fe,1,1,0,62,5950.0,3307.5,630.0
25100,Lomas de Chapultepec,1,1,1,50,15050.0,6055.0,906.5
25103,Condesa,1,0,0,50,6825.0,1575.0,245.0


In [65]:
seleccion2 = df['Valor'] < 4200
df[seleccion2]

,Colonia,Habitaciones,Garages,Suites,Area,Valor,Condominio,Impuesto
3,Centro Histórico,1,0,0,15,2800.0,1365.0,70.0
4,Del Valle,1,0,0,48,2800.0,805.0,0.0
19,Narvarte,1,0,0,48,2625.0,2702.0,87.5
37,Centro Histórico,1,0,0,27,2625.0,1298.5,66.5
61,Centro Histórico,1,1,0,37,3850.0,1890.0,3150.0
...,...,...,...,...,...,...,...,...
25038,Villa de Guadalupe,2,1,0,50,2800.0,1435.0,0.0
25051,Nativitas,2,1,0,62,3500.0,1050.0,0.0
25065,Nativitas,2,1,0,50,3150.0,1050.0,0.0
25095,Lindavista Sur,2,0,0,63,2800.0,1372.0,0.0


In [66]:
# para que se cumplan las dos condiciones anteriores
filtro1 = (seleccion1) & (seleccion2)
df[filtro1]

,Colonia,Habitaciones,Garages,Suites,Area,Valor,Condominio,Impuesto
3,Centro Histórico,1,0,0,15,2800.0,1365.0,70.0
4,Del Valle,1,0,0,48,2800.0,805.0,0.0
19,Narvarte,1,0,0,48,2625.0,2702.0,87.5
37,Centro Histórico,1,0,0,27,2625.0,1298.5,66.5
61,Centro Histórico,1,1,0,37,3850.0,1890.0,3150.0
...,...,...,...,...,...,...,...,...
24710,Nativitas,1,0,0,40,3150.0,1522.5,0.0
24872,Peralvillo,1,0,1,69,3150.0,2240.0,441.0
24880,La Condesa,1,0,0,46,3850.0,1585.5,112.0
24943,Roma,1,0,0,21,3500.0,987.0,87.5


In [68]:
df1 = df[filtro1]

In [71]:
# 2. minimo 2 dormitorios, alquiler menor a 10500 y superficie mayor a 70m2
filtro2 = (df['Habitaciones'] >= 2) & (df['Valor'] < 10500) & (df['Area'] > 70)
df2 = df[filtro2]
df2

,Colonia,Habitaciones,Garages,Suites,Area,Valor,Condominio,Impuesto
14,Narvarte,2,1,0,110,6650.0,2450.0,483.0
16,Narvarte,2,1,0,78,7000.0,2450.0,0.0
21,Roma,2,1,0,76,8750.0,2590.0,0.0
33,Santa Fe,3,1,1,72,8225.0,2100.0,245.0
58,Santa Fe,3,2,1,104,9100.0,4774.0,1421.0
...,...,...,...,...,...,...,...,...
25028,Del Valle,2,0,1,90,7700.0,1890.0,483.0
25062,Santa Fe,2,1,1,72,5950.0,3045.0,560.0
25063,Narvarte,3,1,0,75,6650.0,3010.0,493.5
25105,Santa Fe,2,1,1,85,7875.0,5463.5,689.5


# Exportando datos
- DataFrame completo
- DataFrame con el filtro1
- DataFrame con el filtro2

In [ ]:
# Guardando
df.to_csv('exported-data/inmuebles_ml.csv')

# Leyendo archivo para comprobar
pd.read_csv("exported-data/inmuebles_ml.csv")

,Unnamed: 0,Colonia,Habitaciones,Garages,Suites,Area,Valor,Condominio,Impuesto
0,3,Centro Histórico,1,0,0,15,2800.0,1365.0,70.0
1,4,Del Valle,1,0,0,48,2800.0,805.0,0.0
2,8,Condesa,1,0,1,40,7000.0,1963.5,175.0
3,10,Condesa,4,3,1,243,45500.0,7000.0,2810.5
4,13,Santa Fe,2,1,1,67,5950.0,2061.5,514.5
...,...,...,...,...,...,...,...,...,...
14259,25111,Lomas de Chapultepec,3,0,1,80,10500.0,3535.0,871.5
14260,25113,Coyoacán,3,1,2,150,52500.0,4900.0,2100.0
14261,25114,Narvarte,1,0,0,28,3500.0,1260.0,87.5
14262,25118,Mixcoac,2,1,0,48,4900.0,1781.5,129.5


En este punto aparece una columna `Unnamed` que hace referencia a los indices antiguos, que ya no nos sirven. Para solucionar esto es necesario agregar un argumento `index=False`.
Por otro lado, si se quisiera cambiar el separador (por defecto la coma) y la tabla entregada en un principio ocupaba el punto y coma, para arreglarlo se utilizará el argumento `sep=';'`

In [78]:
df.to_csv("exported-data/inmuebles_ml.csv", index=False, sep=';')

pd.read_csv("exported-data/inmuebles_ml.csv")

,Colonia;Habitaciones;Garages;Suites;Area;Valor;Condominio;Impuesto
0,Centro Histórico;1;0;0;15;2800.0;1365.0;70.0
1,Del Valle;1;0;0;48;2800.0;805.0;0.0
2,Condesa;1;0;1;40;7000.0;1963.5;175.0
3,Condesa;4;3;1;243;45500.0;7000.0;2810.5
4,Santa Fe;2;1;1;67;5950.0;2061.5;514.5
...,...
14259,Lomas de Chapultepec;3;0;1;80;10500.0;3535.0;8...
14260,Coyoacán;3;1;2;150;52500.0;4900.0;2100.0
14261,Narvarte;1;0;0;28;3500.0;1260.0;87.5
14262,Mixcoac;2;1;0;48;4900.0;1781.5;129.5


In [80]:
# Guardando el resto de los datos
df1.to_csv('exported-data/inmuebles_ml_filtro1.csv', index=False, sep=';')
df2.to_csv("exported-data/inmuebles_ml_filtro2.csv", index=False, sep=";")